In [4]:
# an agent to help perform searches: for financial news, sentiment, etc.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_tavily import TavilySearch

from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
tool = TavilySearch(
    api_key="TAVILY_API_KEY",
    search_type="news",
    language="en",
    max_results=5,
    sort_by="relevancy",
)

In [29]:
chat = ChatGroq(
    temperature=0.5,
    model='deepseek-r1-distill-llama-70b'
)

prompt = ChatPromptTemplate.from_messages(
    messages=[
        (
            "system", '''
                        You are a stock and financial analyst, who helps with financial news searches, and helps to analyze the sentiment of the news you find.
                        You can search for news articles, and then analyze the sentiment of the articles you find using the tools given to you.
                        You can also use the search results to help answer questions about the news articles you find.

                        You may use news sources like Yahoo Finance, Google Finance, MarketWatch, Wall Street Journal, and others.

                        Provide your analysis in a clear and concise manner, and make sure to include the most relevant information from the articles you find, and always link to the sources you use.
                    '''
        ),
        ("user", "I want to search for financial news about {search_term}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(
    llm=chat,
    tools=[tool],
    prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=[tool], verbose=True)

In [30]:
response = agent_executor.invoke(
    {"search_term": "financial news about Apple"},
    run_name="financial-news-search",
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'financial news about Apple', 'search_depth': 'advanced', 'topic': 'finance'}`


{'query': 'financial news about Apple', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://finance.yahoo.com/quote/AAPL/news/', 'title': 'Apple Inc. (AAPL)', 'content': "Breaking News • 22 hours ago Stocks resume slide after two-day rally with Fed in focus News • 3 days ago A Fed meeting and retail sales data greet a flailing stock market: What to know this week News • 3 days ago How Trump has talked about stock market gyrations since his election win Breaking News • 4 days ago The S&P 500 jumped nearly 2% after it entered into a correction, as stocks were set to end another volatile week on a high note. Breaking News • 4 days ago Dow, S&P 500, Nasdaq jump but stocks head for sharp weekly losses Breaking News • 5 days ago The S&P 500 headed for correction territory as stocks sank amid Pre

In [31]:
response['output']

'Based on the search results, here is a concise analysis of the financial news about Apple:\n\n1. **Stock Performance**: Apple\'s stock (AAPL) has experienced a significant decline, shedding nearly $700 billion in market value since its record high in December. The stock has dropped approximately 18% from its peak and is currently trading around $188.26 per share, which is 27.3% below its 52-week high of $259.02.\n\n2. **Market Impact**: The recent sell-off, partly fueled by tariff concerns, has led to Apple\'s stock being on track for its worst performance since March 2020. The stock plummeted over 8% in afternoon trade, erasing $300 billion from its market capitalization.\n\n3. **Analyst Insights**: Despite the downturn, Tigress Financial analyst Ivan Feinseth has maintained a "Strong Buy" rating on Apple and raised the price target from $295 to $300, indicating optimism about the company\'s long-term prospects.\n\nFor more detailed information, you can visit the following sources:\n